In [1]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
%matplotlib inline
import seaborn as sns
from scipy import stats
import statsmodels.api as sm
from sklearn.model_selection import train_test_split

from sklearn.ensemble import RandomForestRegressor
from sklearn.decomposition import PCA, SparsePCA

from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline

from sklearn.datasets import make_regression
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedKFold
from sklearn.ensemble import GradientBoostingRegressor

from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import MinMaxScaler
from sklearn.decomposition import PCA
from sklearn.preprocessing import LabelEncoder


## Test Data Cleaning

In [3]:
#test_clean.csv came from test.csv
test = pd.read_csv('/Users/mercuryliu/Documents/Kaggle/ga-customer-revenue-prediction/test_v2_withid.csv', low_memory=False).drop(['Unnamed: 0'], axis=1)

In [4]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 401589 entries, 0 to 401588
Data columns (total 27 columns):
 #   Column                                    Non-Null Count   Dtype  
---  ------                                    --------------   -----  
 0   visitNumber                               401589 non-null  int64  
 1   totals.visits                             401589 non-null  int64  
 2   totals.hits                               401589 non-null  int64  
 3   totals.pageviews                          401488 non-null  float64
 4   totals.timeOnSite                         217903 non-null  float64
 5   totals.newVisits                          286065 non-null  float64
 6   totals.transactions                       6305 non-null    float64
 7   totals.totalTransactionRevenue            4594 non-null    float64
 8   totals.bounces                            182678 non-null  float64
 9   channelGrouping                           401589 non-null  object 
 10  socialEngagementType

In [5]:
ID = test['fullVisitorId']

In [6]:
test_x = test.drop(['fullVisitorId'], axis=1)

In [7]:
test_x.shape

(401589, 26)

In [8]:
test_x.head()

,visitNumber,totals.visits,totals.hits,totals.pageviews,totals.timeOnSite,totals.newVisits,totals.transactions,totals.totalTransactionRevenue,totals.bounces,channelGrouping,...,geoNetwork.subContinent,geoNetwork.country,geoNetwork.networkDomain,trafficSource.campaign,trafficSource.source,trafficSource.medium,trafficSource.keyword,trafficSource.adwordsClickInfo.isVideoAd,trafficSource.referralPath,trafficSource.isTrueDirect
0,2,1,4,3.0,973.0,NaN,NaN,NaN,NaN,Organic Search,...,Southern Asia,India,unknown.unknown,(not set),google,organic,(not provided),NaN,(not set),True
1,166,1,4,3.0,49.0,NaN,NaN,NaN,NaN,Direct,...,Northern America,United States,(not set),(not set),(direct),(none),(not set),NaN,(not set),True
2,2,1,4,3.0,24.0,NaN,NaN,NaN,NaN,Organic Search,...,Northern America,United States,onlinecomputerworks.com,(not set),google,organic,(not provided),NaN,(not set),True
3,4,1,5,4.0,25.0,NaN,NaN,NaN,NaN,Direct,...,Northern America,United States,(not set),(not set),(direct),(none),(not set),NaN,(not set),True
4,1,1,5,4.0,49.0,1.0,NaN,NaN,NaN,Organic Search,...,Northern America,United States,com,(not set),google,organic,(not provided),NaN,(not set),NaN


In [9]:
#an useful function, help decide which columns to drop due to missing values
def Null_Count(df):
    df_null = df.isnull().sum().sort_values(ascending = False).rename('Null').reset_index()

    null_count = df_null['Null']
    null_percent = (null_count * 100) / (df.shape[0])

    df_null = pd.concat([df_null['index'],null_count,null_percent], axis=1, keys=['Column','Null_Count','Null_Percent'])

    return df_null[df_null['Null_Count'] != 0]

Null_Count(test_x)

,Column,Null_Count,Null_Percent
0,totals.totalTransactionRevenue,396995,98.856044
1,totals.transactions,395284,98.429987
2,trafficSource.adwordsClickInfo.isVideoAd,390984,97.359240
3,trafficSource.isTrueDirect,253180,63.044556
4,totals.bounces,218911,54.511204
5,totals.timeOnSite,183686,45.739799
6,totals.newVisits,115524,28.766724
7,trafficSource.keyword,40226,10.016709
8,totals.pageviews,101,0.025150


In [10]:
def fill_na(df):   
    df["totals.totalTransactionRevenue"].fillna(0.0, inplace=True)
    df["totals.transactions"].fillna(0.0, inplace=True)
    df['totals.bounces'].fillna(0, inplace=True) 
    df['totals.timeOnSite'].fillna(0, inplace=True)
    df['totals.newVisits'].fillna(0, inplace=True)
    df['totals.pageviews'].fillna(1, inplace=True)

    
    # Changing datatypes from object to desired ones
    df['totals.pageviews'] = df['totals.pageviews'].astype(int)
    df['totals.newVisits'] = df['totals.newVisits'].astype(int)
    df['totals.bounces'] = df['totals.bounces'].astype(int)
    df["totals.totalTransactionRevenue"] = df["totals.totalTransactionRevenue"].astype(float)
    df["totals.transactions"] = df["totals.totalTransactionRevenue"].astype(float)

    
    df['trafficSource.isTrueDirect'].fillna(False, inplace=True) 
    df['trafficSource.adwordsClickInfo.isVideoAd'].fillna(True, inplace=True)
    df['trafficSource.keyword'].fillna('(not provided)', inplace=True) 

    return df

test_x = fill_na(test_x)

In [11]:
test_x.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 401589 entries, 0 to 401588
Data columns (total 26 columns):
 #   Column                                    Non-Null Count   Dtype  
---  ------                                    --------------   -----  
 0   visitNumber                               401589 non-null  int64  
 1   totals.visits                             401589 non-null  int64  
 2   totals.hits                               401589 non-null  int64  
 3   totals.pageviews                          401589 non-null  int64  
 4   totals.timeOnSite                         401589 non-null  float64
 5   totals.newVisits                          401589 non-null  int64  
 6   totals.transactions                       401589 non-null  float64
 7   totals.totalTransactionRevenue            401589 non-null  float64
 8   totals.bounces                            401589 non-null  int64  
 9   channelGrouping                           401589 non-null  object 
 10  socialEngagementType

In [12]:
num_name = ['visitNumber', 'totals.visits', 'totals.hits', \
            'totals.pageviews', 'totals.timeOnSite', 'totals.newVisits', \
            'totals.transactions', 'totals.totalTransactionRevenue', 'totals.bounces']
num_c = test_x[num_name]
cat_c = test_x.drop(num_name, axis=1)

In [13]:
for i in range(len(cat_c.columns)):
    cat_c.iloc[:,i] = LabelEncoder().fit_transform(cat_c.iloc[:,i])

In [14]:
X_total = pd.concat([num_c, cat_c], axis=1)

In [15]:
real_x = pd.read_csv('/Users/mercuryliu/Documents/Kaggle/ga-customer-revenue-prediction/X_v2.csv', \
                low_memory=False).drop(['visitHour', 'Unnamed: 0'], axis=1)

In [16]:
X_total.columns.to_list() == real_x.columns.to_list()

True

In [17]:
X_total.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 401589 entries, 0 to 401588
Data columns (total 26 columns):
 #   Column                                    Non-Null Count   Dtype  
---  ------                                    --------------   -----  
 0   visitNumber                               401589 non-null  int64  
 1   totals.visits                             401589 non-null  int64  
 2   totals.hits                               401589 non-null  int64  
 3   totals.pageviews                          401589 non-null  int64  
 4   totals.timeOnSite                         401589 non-null  float64
 5   totals.newVisits                          401589 non-null  int64  
 6   totals.transactions                       401589 non-null  float64
 7   totals.totalTransactionRevenue            401589 non-null  float64
 8   totals.bounces                            401589 non-null  int64  
 9   channelGrouping                           401589 non-null  int64  
 10  socialEngagementType

In [18]:
X_unique = X_total
X_unique['fullVisitorId'] = ID

In [19]:
X_unique.drop_duplicates(subset ='fullVisitorId',
                     keep = 'first', inplace = True)
  

In [20]:
X_noid = X_unique.drop('fullVisitorId', axis = 1)

In [21]:
scaler = MinMaxScaler()
X_s = scaler.fit_transform(X_noid)

pca_n = PCA(n_components = 12)
X_r = pca_n.fit_transform(X_s)

In [22]:
np.sum(pca_n.explained_variance_ratio_)> 0.95

True

## Model Fitting

In [23]:
X = real_x
y = pd.read_csv('/Users/mercuryliu/Documents/Kaggle/ga-customer-revenue-prediction/y_v2.csv').drop('Unnamed: 0', axis=1)


In [24]:
y_log = np.log(y + 1)
X_train, X_test, y_train, y_test = train_test_split(X, y_log, test_size=0.3, random_state=42)

In [25]:
#remeber to reshape y to 1d array, otherwise warning message would be raised
y_train = y_train.to_numpy().reshape(len(y_train),)

In [26]:
scaler = MinMaxScaler()
X_train_s = scaler.fit_transform(X_train)

pca_train = PCA(n_components = 12)
X_train_r = pca_train.fit_transform(X_train_s)

In [27]:
m_best = GradientBoostingRegressor(n_estimators=70, max_depth=20, min_samples_leaf=20,\
                                  loss='ls', learning_rate=0.05, criterion='mse',\
                                  max_features='sqrt', random_state=2)

In [28]:
m_best.fit(X_train_r, y_train) #fit model


GradientBoostingRegressor(criterion='mse', learning_rate=0.05, max_depth=20,
                          max_features='sqrt', min_samples_leaf=20,
                          n_estimators=70, random_state=2)

## Prediction

In [29]:
pred = m_best.predict(X_r)


## Submission

In [30]:
sample = pd.read_csv('/Users/mercuryliu/Documents/Kaggle/ga-customer-revenue-prediction/sample_submission_v2.csv',\
                    low_memory=False).drop('PredictedLogRevenue', axis=1)

In [31]:
sample['fullVisitorId']

Index(['fullVisitorId'], dtype='object')

In [32]:
submission = pd.DataFrame()
submission['fullVisitorId'] = X_unique['fullVisitorId']
submission['PredictedLogRevenue'] = pred

In [33]:
submission['new_id'] = sample['fullVisitorId']

In [34]:
submission.head()

,fullVisitorId,PredictedLogRevenue,new_id
0,7460955084541987166,0.436980,0000018966949534117
1,460252456180441002,1.282965,0000039738481224681
2,3461808543879602873,0.082952,0000073585230191399
3,975129477712150630,0.981545,0000087588448856385
4,8381672768065729990,-0.041743,0000149787903119437


In [40]:
submission.sort_values(by=['fullVisitorId'], inplace=True)

In [41]:
submission2 = submission.drop('new_id', axis=1)

In [42]:
submission2.head()

,fullVisitorId,PredictedLogRevenue
264441,0000018966949534117,0.080926
187425,0000039738481224681,-0.002738
91862,0000073585230191399,0.000631
23103,0000087588448856385,0.023654
303019,0000149787903119437,-0.004338


In [ ]:
submission.to_csv(r'/Users/mercuryliu/Documents/Kaggle/ga-customer-revenue-prediction/First Submission.csv',\
                 index=False)

In [43]:
submission2.to_csv(r'/Users/mercuryliu/Documents/Kaggle/ga-customer-revenue-prediction/Submission2.csv',\
                 index=False)

In [44]:
s = pd.read_csv('/Users/mercuryliu/Documents/Kaggle/ga-customer-revenue-prediction/Submission2.csv', \
               low_memory=False)

In [45]:
s.columns

Index(['fullVisitorId', 'PredictedLogRevenue'], dtype='object')

In [46]:
s.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 296530 entries, 0 to 296529
Data columns (total 2 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   fullVisitorId        296530 non-null  object 
 1   PredictedLogRevenue  296530 non-null  float64
dtypes: float64(1), object(1)
memory usage: 4.5+ MB


In [47]:
sample2 = pd.read_csv('/Users/mercuryliu/Documents/Kaggle/ga-customer-revenue-prediction/sample_submission_v2.csv',\
                    low_memory=False)

In [48]:
sample2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 296530 entries, 0 to 296529
Data columns (total 2 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   fullVisitorId        296530 non-null  object 
 1   PredictedLogRevenue  296530 non-null  float64
dtypes: float64(1), object(1)
memory usage: 4.5+ MB
